In [1]:
# If file not downloaded
import zipfile
import pandas as pd
with zipfile.ZipFile("archive.zip", "r") as zip_ref:
    zip_ref.extractall(".")


In [3]:
# create folder
import os
import time
from datetime import datetime
timenow = datetime.now().strftime('%x,%X')

class extract:
    def __init__(self, input):
        data = input.split(',')
        self.date = data[0].split('/')
        self.time = data[1].split(':')
    def day(self):
        return int(self.date[1])
    def month(self):
        return int(self.date[0])
    def year(self):
        return int(self.date[2])
    def hour(self):
        return int(self.time[0])
    def minute(self):
        return int(self.time[1])
    def second(self):
        return int(self.time[2])
    


    
current = os.getcwd()
folder = "\\theatre"
os.mkdir(current+folder)
if os.path.getsize(current+folder) == 0:
    for filename in os.listdir(current):
        f = os.path.join(current, filename)
#         print(f)
#         print(current+folder)
        modtime = time.strftime('%x,%X', time.localtime(os.path.getmtime(f)))
        filetime = extract(modtime)
        nowtime = extract(timenow)
        if filetime.year() == nowtime.year() and filetime.month() == nowtime.month() and filetime.day() == nowtime.day():
#             print(f,current+folder)
#             print(filetime.day(),nowtime.day())
#             print(filetime.month(),nowtime.month())
#             print(filetime.year(),nowtime.year())
            if f != current+folder:
                os.rename(current+'\\'+filename, current+folder+'\\'+filename)
os.chdir(current+folder)




        
        
        



FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\leezh\\Downloads\\theatre'

In [6]:
os.chdir(current+folder)

In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

num_classes = 0
for i in os.listdir(os.getcwd()):
    num_classes += 1

# Specify the path to the 'theatre' folder
data_folder = os.getcwd()

# Initialize empty lists to store the images and their corresponding labels
images = []
labels = []

# Iterate through each genre folder
for genre in os.listdir(data_folder):
    genre_folder = os.path.join(data_folder, genre)
    if os.path.isdir(genre_folder):
        # Iterate through each image file in the genre folder
        for filename in os.listdir(genre_folder):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Load the image using OpenCV
                image = cv2.imread(os.path.join(genre_folder, filename))
                # Preprocess the image (e.g., resize, normalize)
                image = cv2.resize(image, (224, 224))  # Resize to a fixed size
                image = image / 255.0  # Normalize pixel values between 0 and 1
                # Append the image and its genre label to the lists
                images.append(image)
                labels.append(genre)

# Convert the image and label lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Perform label encoding and one-hot encoding
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)


from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # num_classes is the number of movie genres

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))


# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)



Epoch 1/10
246/246 [==============================] - 2514s 10s/step - loss: 2.4587 - accuracy: 0.2955 - val_loss: 2.1445 - val_accuracy: 0.3469
Epoch 2/10
246/246 [==============================] - 2063s 8s/step - loss: 1.8667 - accuracy: 0.4205 - val_loss: 2.2824 - val_accuracy: 0.3250
Epoch 3/10
246/246 [==============================] - 2197s 9s/step - loss: 1.5049 - accuracy: 0.5251 - val_loss: 2.2654 - val_accuracy: 0.3210
Epoch 4/10
246/246 [==============================] - 2264s 9s/step - loss: 1.1279 - accuracy: 0.6545 - val_loss: 2.4484 - val_accuracy: 0.3052
Epoch 5/10
246/246 [==============================] - 2419s 10s/step - loss: 0.7531 - accuracy: 0.7883 - val_loss: 2.6513 - val_accuracy: 0.3128
Epoch 6/10
246/246 [==============================] - 2542s 10s/step - loss: 0.4814 - accuracy: 0.8752 - val_loss: 2.8756 - val_accuracy: 0.3087
Epoch 7/10
246/246 [==============================] - 2336s 10s/step - loss: 0.2921 - accuracy: 0.9410 - val_loss: 3.1130 - val_accur

In [8]:
# Save trained model

# Specify the path and filename for saving the model
save_path = current+folder+'\\model.h5'

# Save the model
model.save(save_path)

from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model(save_path)


In [9]:
# zip up the model
import shutil 
import os.path

# Creating the ZIP file 
archived = shutil.make_archive(save_path, 'zip', save_path)

if os.path.exists(save_path+'.zip'):
   print(archived)
else: 
   print("ZIP file not created")

C:\Users\leezh\Downloads\theatre\model.h5.zip


In [ ]:
# Prediction code
# import just in case it did not get imported
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Specify the path to the folder containing the new movie images
new_movies_folder = 'path/to/new/movies' # this should be replaced with the actual folder path

# Load the trained model
model = load_model('path/to/saved/model.h5') # this should be replaced with the model path

# Initialize an empty list to store the preprocessed movie images
preprocessed_images = []

# Iterate through each movie file in the folder
for filename in os.listdir(new_movies_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the movie image using OpenCV
        movie_image = cv2.imread(os.path.join(new_movies_folder, filename))
        # Preprocess the movie image (e.g., resize, normalize)
        preprocessed_image = cv2.resize(movie_image, (224, 224))  # Resize to the same size as training images
        preprocessed_image = preprocessed_image / 255.0  # Normalize pixel values between 0 and 1
        # Append the preprocessed image to the list
        preprocessed_images.append(preprocessed_image)

# Convert the list of preprocessed images to a NumPy array
preprocessed_images = np.array(preprocessed_images)

# Make predictions on the new movie images
predictions = model.predict(preprocessed_images)

# Assuming one-hot encoded genre labels
# Convert the predictions to genre labels
predicted_labels = [label_encoder.inverse_transform([np.argmax(pred)])[0] for pred in predictions]
